In [1]:
from pathlib import Path
import numpy as np

In [2]:
from util.plot.heatmap import plot_csi

In [3]:
from util.preprocess.csi.denoising import hampel_filter

In [10]:
from util.preprocess.csi.normalize import normalize_csi_data_by_frame_power

In [5]:
from util.preprocess.csi.phase import remove_timing_offset

In [12]:
root_data_dirpath = Path('/tf/workspace/deformation-prediction-multi-environment/data')
resampled_data_dirpath = root_data_dirpath / 'preprocessed' / 'csi' / 'resampled'
save_dirpath = root_data_dirpath / 'preprocessed' / 'csi' / 'real_and_imag'

In [13]:
trash_subcarriers = [0,1,2,3,4,5,32,59,60,61,62,63]
useful_subcarriers = [k for k in range(64) if k not in trash_subcarriers]

In [14]:
def real_preprocess(real):
    real_filtered = hampel_filter(real[:, useful_subcarriers], window_size=200, n_sigmas=3)

    return real_filtered

In [15]:
def imag_preprocess(imag):
    imag_filtered = hampel_filter(imag[:, useful_subcarriers], window_size=200, n_sigmas=3)

    return imag_filtered

In [22]:
def windowing(data, window_size):
    # スライディングウィンドウを適用して、(N, window_size, features)の形に変換
    shape = (data.shape[0] // window_size, window_size, data.shape[1], 2)
    return data[:shape[0] * window_size].reshape(shape)

In [24]:
for location in ['512', '511', '514B']:
    location_data_dirpath = resampled_data_dirpath / location
    
    # サブディレクトリをすべて取得
    session_dirpaths = [d for d in location_data_dirpath.iterdir() if d.is_dir()]
    
    reals = []
    imags = []
    # 結果を表示
    for session_dirpath in session_dirpaths:
        real = np.load(session_dirpath / 'real.npy')
        imag = np.load(session_dirpath / 'imag.npy')
    
        reals.append(real_preprocess(real))
        imags.append(imag_preprocess(imag))

    concatenated_real = np.concatenate(reals)
    concatenated_imag = np.concatenate(imags)

    print(f'{concatenated_real.shape=}')
    print(f'{concatenated_imag.shape=}')

    csi = np.stack([concatenated_real, concatenated_imag], axis=-1)
    print(f'{csi.shape=}')

    normalized_csi = normalize_csi_data_by_frame_power(csi)

    windowed_csi = windowing(normalized_csi, 10)
    print(f'{windowed_csi.shape=}')

    save_location_dirpath = save_dirpath / location
    save_location_dirpath.mkdir(parents=True, exist_ok=True)
    np.save(save_location_dirpath / 'csi.npy', windowed_csi)

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

concatenated_real.shape=(120000, 52)
concatenated_imag.shape=(120000, 52)
csi.shape=(120000, 52, 2)
windowed_csi.shape=(12000, 10, 52, 2)
